#Instalação 

In [ ]:
# reading daaset from Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install emoji
!pip install simplejson
!pip install wmd
!pip install kneed

#BERTopic
!pip install bertopic[all]
!pip install umap-learn

#Metrics
!pip install octis
!pip install tmplot
!pip install git+https://github.com/maximtrp/tmplot.git
!pip install -U contextualized_topic_models

#embedding
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install pyyaml==5.4.1
!pip install flair

In [ ]:
 # Packages to store and manipulate data
import pandas as pd
import numpy as np
# Example function using numpy:
from numpy import dot
from numpy.linalg import norm
import math
import os
import json
import simplejson as json
import csv
import string
import glob
import random
import time
import math
from datetime import datetime
pd.set_option('max_colwidth', 200)
 
# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import pickle
import math
import wmd
from scipy import spatial
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
from collections import Counter
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import tmplot as tmp
from tmplot._helpers import get_phi
from tmplot._helpers import calc_terms_probs_ratio
import umap
from umap import UMAP
 
# Packages
import emoji
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import operator
from operator import itemgetter
from tqdm import tqdm
from collections import defaultdict
from textblob import TextBlob
import sys
import itertools
import spacy
from scipy.sparse import csr_matrix, issparse  # , todense
import sys
from collections import defaultdict
import multiprocessing
from pprint import pprint
from contextualized_topic_models.evaluation.measures import CoherenceNPMI
from hdbscan import HDBSCAN
 
# Gensim
import gensim
import gensim.downloader
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim import models
from gensim import corpora
from gensim.models import CoherenceModel
import gensim.downloader as api
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.models.ldamulticore import LdaMulticore
 
# Model building package: Sklearn
import sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
# Cluster documents: Sklearn
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
 
#BERTopic
#Embeddings
from bertopic import BERTopic
from bertopic.backend import WordDocEmbedder
from sentence_transformers import SentenceTransformer
from flair.embeddings import TransformerDocumentEmbeddings
from flair.embeddings import TransformerWordEmbeddings

#Metrics
from octis.evaluation_metrics.metrics import AbstractMetric
from octis.dataset.dataset import Dataset
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import octis.configuration.citations as citations
import numpy as np
import itertools
from scipy import spatial
from sklearn.metrics import pairwise_distances
from operator import add
from octis.evaluation_metrics.rbo import rbo
import gensim.downloader as api
from gensim.models import KeyedVectors
from octis.evaluation_metrics.diversity_metrics import WordEmbeddingsInvertedRBO, \
    WordEmbeddingsInvertedRBOCentroid, InvertedRBO
from itertools import combinations
from scipy.spatial.distance import cosine
import gensim.downloader as api

#Importar Dados

In [ ]:
df = pd.read_json('/content/drive/dados_fevereiro_dia_de_semana_1.json')

#Tokenizar
new_tweet = []
for tweet in df.clean_tweet:
    tweet_len = [word for word in tweet.split(' ') if len(word) >= 3]
    tweet_string = ' '.join(tweet_len)
    new_tweet.append(tweet_string)
df['clean_tweet'] = new_tweet

stop = stopwords.words('english')
df['clean_tweet_tokenize'] = df['clean_tweet'].apply(lambda x: [item for item in str(x).split(' ')])
df.dropna()

#Modelagem de Tópicos

In [ ]:
# cast tweets to numpy array
docs = df.clean_tweet_tokenize.to_numpy()

# create dictionary of all words in all documents
dictionary =  corpora.Dictionary(docs)

# create variable containing length of dictionary/vocab
vocab_length = len(dictionary)

# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
#UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
# Word embedding
embedding_word_2 = TransformerWordEmbeddings('roberta-large')

# Document embedding model
embedding_document_4 = SentenceTransformer('all-mpnet-base-v2')

# Create a model that uses both language models and pass it through BERTopic
word_doc_embedder = WordDocEmbedder(embedding_model=embedding_document_4, word_embedding_model=embedding_word_2)

In [ ]:
topic_model = BERTopic(nr_topics= "auto", language='english', calculate_probabilities=True, embedding_model=word_doc_embedder, umap_model=umap_model, verbose=True).fit(list(df.clean_tweet))
topics, probabilities = topic_model.fit_transform(list(df.clean_tweet))

In [ ]:
# Save model
topic_model.save("model_train")	
# Load model
topic_model_test = BERTopic.load("model_train")	

In [ ]:
nr_topics = 10
new_topics, new_probs = topic_model.reduce_topics(df.clean_tweet, topics, probabilities, nr_topics=nr_topics)
freq = topic_model.get_topic_info();

#Tópicos

In [ ]:
topic = pd.DataFrame()
topics_train = []
for i in range(1, nr_topics + 1):
  top = []
  name = 'Topic '+ str(i - 1)
  topic_nr = freq.iloc[i]["Topic"]
  for j in range(len(topic_model.get_topic(topic_nr))):
    top.append(topic_model.get_topic(topic_nr)[j][0])
  topics_train.append(top)
  topic[name] = top
topic 

#Métricas

In [ ]:
cm_bertopic = CoherenceModel(topics=topics_train, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs, 
                          coherence='c_v')

print(cm_bertopic.get_coherence()) 

In [ ]:
npmi = CoherenceModel(topics=topics_train, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs,
                          coherence='c_npmi')

print(npmi.get_coherence())

In [ ]:
def get_word2index(list1, list2):
    words = set(list1)
    words = words.union(set(list2))
    word2index = {w: i for i, w in enumerate(words)}
    return word2index

In [ ]:
#Diversity Metrics:
class TopicDiversity(AbstractMetric):
    def __init__(self, topk=10):
        """
        Initialize metric
        Parameters
        ----------
        topk: top k words on which the topic diversity will be computed
        """
        AbstractMetric.__init__(self)
        self.topk = topk

    def score(self, model_output):
        """
        Retrieves the score of the metric
        Parameters
        ----------
        model_output : dictionary, output of the model
                       key 'topics' required.
        Returns
        -------
        td : score
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than ' + str(self.topk))
        else:
            unique_words = set()
            for topic in topics:
                unique_words = unique_words.union(set(topic[:self.topk]))
            td = len(unique_words) / (self.topk * len(topics))
            return td

In [ ]:
class InvertedRBO(AbstractMetric):
    def __init__(self, topk=10, weight=0.9):
        """
        Initialize metric Inverted Ranked-Biased Overlap
        :param topk: top k words on which the topic diversity will be computed
        :param weight: weight of each agreement at depth d. When set to 1.0, there is no weight, the rbo returns to
        average overlap. (default 0.9)
        """
        super().__init__()
        self.topk = topk
        self.weight = weight

    def score(self, model_output):
        """
        Retrieves the score of the metric
        :param model_output : dictionary, output of the model. the 'topics' key is required.
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')
        else:
            collect = []
            for list1, list2 in itertools.combinations(topics, 2):
                word2index = get_word2index(list1, list2)
                indexed_list1 = [word2index[word] for word in list1]
                indexed_list2 = [word2index[word] for word in list2]
                rbo_val = rbo(indexed_list1[:self.topk], indexed_list2[:self.topk], p=self.weight)[2]
                collect.append(rbo_val)
            return 1 - np.mean(collect)

In [ ]:
#Similarity Metrics:
class PairwiseJaccardSimilarity(AbstractMetric):
    def __init__(self, topk=10):
        """
        Initialize metric Pairwise Jaccard Similarity
        Parameters
        ----------
        :param topk: top k words on which the topic diversity will be computed
        """
        super().__init__()
        self.topk = topk

    def score(self, model_output):
        """
        Retrieves the score of the metric
        :return PJS
        """
        topics = model_output
        sim = 0
        count = 0
        for list1, list2 in combinations(topics, 2):
            intersection = len(list(set(list1[:self.topk]).intersection(list2[:self.topk])))
            union = (len(list1[:self.topk]) + len(list2[:self.topk])) - intersection
            count = count + 1
            sim = sim + (float(intersection) / union)
        return sim / count

In [ ]:
diversity = TopicDiversity()
diversity.score(topics_train)

In [ ]:
b = InvertedRBO()
b.score(topics_train)

In [ ]:
jaccard = PairwiseJaccardSimilarity()
jaccard.score(topics_train)

##Visualization

In [ ]:
representative_docs = topic_model.get_representative_docs(5)
representative_docs #Para obter os documentos representativos de um único tópico

In [ ]:
fig = topic_model.visualize_topics(); fig

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=df.clean_tweet, 
                                                topics=new_topics, 
                                                timestamps=df.created_at, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=8)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=11)

In [ ]:
topic_model.visualize_barchart(top_n_topics=11)